### Count number of non-duplicate, uniquely mapped exonic reads

```
Expected from TEST.sorted.bam:
uniqMappedNonDupeReadCount = 986
estExonicUniqMappedNonDupeReadCount 854.78
```

In [211]:
import pybedtools
import multiprocessing

In [207]:
threads = max(multiprocessing.cpu_count(), 16) # Don't swamp stacker

In [ ]:
sample_ids = ["TH27_0699_S02", "THR29_0754_S01", "TH02_0093_S01"]

In [168]:
!sambamba flagstat TEST.sorted.bam

2642 + 0 in total (QC-passed reads + QC-failed reads)
644 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2618 + 0 mapped (99.09%:N/A)
1998 + 0 paired in sequencing
999 + 0 read1
999 + 0 read2
1974 + 0 properly paired (98.80%:N/A)
1974 + 0 with itself and mate mapped
0 + 0 singletons (0.00%:N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [210]:
# Filter to get unique reads
!sambamba markdup --remove-duplicates --threads {threads} TEST.sorted.bam data/TEST.dedupe.bam
!sambamba view --valid \
    --nthreads {threads} \
    --filter "not (failed_quality_control or secondary_alignment or supplementary or unmapped)" \
    data/TEST.dedupe.bam -f bam > data/TEST.filtered.bam
!sambamba flagstat data/TEST.filtered.bam

sambamba-markdup: Unrecognized option --threads
1972 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
1972 + 0 mapped (100.00%:N/A)
1972 + 0 paired in sequencing
986 + 0 read1
986 + 0 read2
1972 + 0 properly paired (100.00%:N/A)
1972 + 0 with itself and mate mapped
0 + 0 singletons (0.00%:N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [192]:
bed = pybedtools.BedTool("hg38_GENCODE_v23_basic.bed.gz")
bam = pybedtools.BedTool("data/TEST.filtered.bam")

In [212]:
# Single threaded intersect
exonic_reads = bam.intersect(bed)
print("Exonic Reads", exonic_reads.count())

Exonic Reads 1850
